In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
dataset = pd.read_csv('pipe.csv')
dataset.head()

,material,tomr,cf,c_wave_no,cd_radial,cw_circumferencial,crack
0,3,0.4217,2,1,0.6131,0.9991,1
1,6,0.7629,4,-2,0.3846,0.9982,1
2,5,0.1898,1,-7,0.6917,0.9982,1
3,2,0.4062,2,-2,0.5927,0.9980,1
4,6,0.8078,4,-10,0.9345,0.9980,1


In [4]:
dataset.isnull().sum()

material              0
tomr                  0
cf                    0
c_wave_no             0
cd_radial             0
cw_circumferencial    0
crack                 0
dtype: int64

In [6]:
X = dataset.drop(['crack'] , axis=1).to_numpy()
X

array([[ 3.    ,  0.4217,  2.    ,  1.    ,  0.6131,  0.9991],
       [ 6.    ,  0.7629,  4.    , -2.    ,  0.3846,  0.9982],
       [ 5.    ,  0.1898,  1.    , -7.    ,  0.6917,  0.9982],
       ...,
       [ 3.    ,  0.0448,  1.    , -2.    ,  0.5649,  0.0254],
       [ 1.    ,  0.7149,  3.    ,  9.    ,  0.2688,  0.0253],
       [ 4.    ,  0.7759,  4.    ,  7.    ,  0.9258,  0.0252]])

In [7]:
Y = dataset.crack.to_numpy()
Y

array([1, 1, 1, ..., 0, 1, 1])

In [8]:
from sklearn.preprocessing import StandardScaler
s = StandardScaler()
x3 = s.fit_transform(X)

In [9]:
from imblearn.over_sampling import SMOTE
s=SMOTE()
x1,y1=s.fit_resample(x3,Y)

In [11]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x1,y1,random_state=10,test_size=0.2)

In [12]:
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

sv=SVC(kernel="linear")
kn=KNeighborsClassifier(n_neighbors=5,metric="minkowski",p=2)
r=RandomForestClassifier()

sv.fit(x_train,y_train)
kn.fit(x_train,y_train)
r.fit(x_train,y_train)

RandomForestClassifier()

In [13]:
p1=sv.predict(x_train)
p2=kn.predict(x_train)
p3=r.predict(x_train)

In [14]:
print("SVM",p1)
print("KNN",p2)
print("RandomF",p3)

SVM [1 0 0 ... 0 1 0]
KNN [1 0 0 ... 0 1 0]
RandomF [1 0 1 ... 0 1 0]


In [15]:
from sklearn.ensemble import VotingClassifier
vc=VotingClassifier(estimators=[("SVM",sv),("KNN",kn),("RandomF",r)])
vc.fit(x_train,y_train)

VotingClassifier(estimators=[('SVM', SVC(kernel='linear')),
                             ('KNN', KNeighborsClassifier()),
                             ('RandomF', RandomForestClassifier())])

In [16]:
pp=vc.predict(x_train)
pp

array([1, 0, 0, ..., 0, 1, 0])

In [17]:
from sklearn.metrics import accuracy_score
accuracy_score(pp,y_train)*100

88.95899053627761

In [19]:
from sklearn.metrics import confusion_matrix
confusion_matrix(pp,y_train)

array([[1104,  235],
       [  10,  870]])

In [20]:
from sklearn.metrics import classification_report
classification_report(pp,y_train)

'              precision    recall  f1-score   support\n\n           0       0.99      0.82      0.90      1339\n           1       0.79      0.99      0.88       880\n\n    accuracy                           0.89      2219\n   macro avg       0.89      0.91      0.89      2219\nweighted avg       0.91      0.89      0.89      2219\n'